### 1a) Create merged LULC raster

In [ ]:
import os
print(os.getcwd())
print(os.listdir(r"C:\temp\timor_leste"))


In [ ]:
import glob
import numpy as np
import rasterio
from rasterio.merge import merge

# Paths
input_dir = r"C:\temp\timor_leste"
output_merged_fp = r"C:\temp\timor_leste\merged_reclass.tif"

# 1. Find all TIFFs and open them
tif_files = glob.glob(f"{input_dir}/*.tif")
if not tif_files:
    raise FileNotFoundError(f"No TIFFs found in {input_dir}")
srcs = [rasterio.open(fp) for fp in tif_files]

# 2. Mosaic (merge) all bands into one array
mosaic_arr, mosaic_transform = merge(srcs)

# 3. Prepare output metadata (take CRS, dtype, etc. from the first source)
out_meta = srcs[0].meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height":  mosaic_arr.shape[1],
    "width":   mosaic_arr.shape[2],
    "transform": mosaic_transform,
    "count":   1,
    "dtype":   "uint8"
})

# 4. Reclassify: values 2,3,4 → 1; everything else → 0
#    (mosaic_arr is shape [1, height, width])
reclass = np.where(np.isin(mosaic_arr[0], [2, 3, 4]), 1, 0).astype("uint8")

# 5. Write the single, reclassified tile
with rasterio.open(output_merged_fp, "w", **out_meta) as dst:
    dst.write(reclass, 1)

# 6. Clean up
for src in srcs:
    src.close()

print("✅ Merged & reclassified raster saved to:", output_merged_fp)


### 1b Create merged LULC raster and vector

In [ ]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd

# Paths
merged_raster_fp = r"C:\temp\timor_leste\merged_reclass.tif"
vector_fp        = r"C:\temp\timor_leste\areas_value1.shp"

# 1. Open the merged & reclassified raster
with rasterio.open(merged_raster_fp) as src:
    arr       = src.read(1)
    transform = src.transform
    crs       = src.crs

# 2. Create a boolean mask where value == 1
mask = arr == 1

# 3. Generate polygon shapes for all contiguous areas of 1
shape_gen = shapes(arr, mask=mask, transform=transform)

# 4. Build a GeoDataFrame from those shapes
features = [
    {"properties": {"value": int(v)}, "geometry": geom}
    for geom, v in shape_gen
]
gdf = gpd.GeoDataFrame.from_features(features, crs=crs)

# 5. Save to a shapefile
gdf.to_file(vector_fp)

print("✅ Vector shapefile saved to:", vector_fp)


### 2) Create LULC CSV

In [ ]:
import glob
import pandas as pd

# 1. Define directories
csv_dir = r"C:\temp\timor_leste\csv"
output_fp = r"C:\temp\timor_leste\merged.csv"

# 2. Find all CSV files
csv_files = glob.glob(f"{csv_dir}/*.csv")
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {csv_dir}")

# 3. Read the first file (to capture headers) with only the needed columns
df = pd.read_csv(csv_files[0], usecols=['ADM2_PCODE', 'area_ha', 'class'])

# 4. Read and append the rest (drops their headers automatically)
for f in csv_files[1:]:
    temp = pd.read_csv(f, usecols=['ADM2_PCODE', 'area_ha', 'class'])
    df = pd.concat([df, temp], ignore_index=True)

# 5. Write out a single merged CSV (header only once)
df.to_csv(output_fp, index=False)

print("✅ Merged CSV saved to:", output_fp)
